In [49]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import itertools
import re

In [19]:
data = pd.read_excel("C:/Users/Administrator/Desktop/word_dealing/data20180521.xlsx",encoding = 'gbk')

In [37]:
df = pd.read_excel("C:/Users/Administrator/Desktop/RFQ collectin .xls",sheetname=1).iloc[:,1:]

In [38]:
product = df[df.iloc[:,0] == "Product Detail Description"].iloc[:,1].values

In [39]:
def translate_lower(product_words):
        '''把大写的字母转化为小写的,有可能还有数字
        product_words是一个产品分割字符串后的列表'''
        for index,word in enumerate(product_words):
            try:
                product_words[index] = word.lower()
            except AttributeError:
                continue
        return product_words

In [40]:
def filter_words(list_tuple_couple):
        '''list_tuple_couple是元组对列表.......[('a','NN'),('b','NJ')] '''
        # 循环剔除介词，组词等
        words_list = []  # 返回的数据列表
        for index, tuples in enumerate(list_tuple_couple):
            filter_words_type = ["IN", "JJ", "CC","CD","POS", "JJR", "JJS","RB","RBS","RBR","RP","VBG","VB","VBD","VBP","VBZ",
                                 "DT","MD",'PDT',"#","$",",",".",":"]  # 想要剔除的数据类型
            # 选择未在filter_words_type中的数据
            if list_tuple_couple[index][1] not in filter_words_type:
                word = list_tuple_couple[index][0]
                words_list.append(word)
        return words_list

In [42]:
disease_List = nltk.word_tokenize(product[0])  # 产生出的分词
# 去除重复的分词，并且按照原来的顺序排列
if len(np.unique(disease_List)) != len(disease_List):
    new_disease_List = list(set(disease_List))
    new_disease_List.sort(key=disease_List.index)
else:
    new_disease_List = disease_List

filtered = [w for w in new_disease_List if w not in stopwords.words('english')]  # 去除停用词汇
filtered = translate_lower(filtered)
Rfiltered = nltk.pos_tag(filtered)  # 标记每一个词汇的词性
words_list = filter_words(Rfiltered)  # 调用帅选想要的单词

In [43]:
#选择有专属名词
word_filtered = []
for t_c in Rfiltered:
    if t_c[1] in ["NN","NNS","NNP","NNPS"]:
        word_filtered.append(t_c[0])
word_filtered = word_filtered[::-1][:3]

In [50]:
index_n = []    #选择商家的index
w_thea = [2/3.0,1/4.0,1/12.0]
#寻找产品的
for i,w in enumerate(data["PRODUCT"]):
    w_l = w.lower()
    thea_sum = 0   #每一个商品的权重
    for i_list,N in enumerate(word_filtered):
        #用正则匹配，例子：因为 light 能匹配 flight
        pattern = re.compile("^"+N+"| "+N)
        if pattern.search(w_l) is not None:
            thea_sum+=w_thea[i_list]
    if thea_sum > 0.5:
        index_n.append(i)                 

In [51]:
data.iloc[index_n,[6,7]].to_csv("C:/Users/Administrator/Desktop/result_supplier.csv",encoding = 'gbk')

In [46]:
df

,Unnamed: 1,Unnamed: 2
0,NaN,NaN
1,Request For Quotation List,NaN
2,Contact Information,NaN
3,First Name,Eugeniusz
4,Last Name,Borowka
5,E-mail,info@bowi.com.pl
6,Company Name,PHU Bowi
7,Country,Poland
8,Product Information,NaN
9,Product Name,LED Lighting


In [48]:
Rfiltered

[(u'led', 'VBN'), (u'lighting', 'NN')]